<a href="https://colab.research.google.com/github/bartala/PCLvsCAPS/blob/main/Reduced_PCL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from io import StringIO
from sklearn.metrics import roc_curve, auc


PTH = '/content/drive/MyDrive/Colab Notebooks/Postpartum_depression/Reduced_PCL/data'

In [ ]:
df = pd.read_excel(f'{PTH}/PCL_CAPS_ALL.xlsx')
df = df.iloc[:,2:23]
df['PCL_total'] = df.iloc[:,0:20].sum(axis=1)
df['PCL_reduced_total'] = df[["pcl_q10","pcl_q2","pcl_q7","pcl_q15","pcl_q4","pcl_q9"]].sum(axis=1)
df

In [3]:
print(df['PCL_total'].mean(),
",",
df['PCL_total'].max(),
",",
df['PCL_total'].min()
)

18.588785046728972 , 57 , 0


In [4]:
# Calc AUC with no ML model

import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

# CAPS: 0/1 ground truth, PCL5: total score 0–80 (float/int)
CAPS = df['caps_ptsd1']
PCL5 = df['PCL_total'] #PCL_total
#PCL5 = df['PCL_reduced_total']

CAPS = np.asarray(CAPS, dtype=float)
PCL5 = np.asarray(PCL5, dtype=float)

# drop NaNs if any
mask = ~np.isnan(CAPS) & ~np.isnan(PCL5)
y = CAPS[mask]
x = PCL5[mask]

# If higher PCL-5 means more likely PTSD (it does), use as-is:
auc_val = roc_auc_score(y, x)
fpr, tpr, thresholds = roc_curve(y, x)   # for plotting if needed

print("AUC:", auc_val)


AUC: 0.9231481481481482


In [5]:
import numpy as np
from sklearn.metrics import roc_auc_score

y = CAPS[mask].astype(int)
x = PCL5[mask].astype(float)

auc = roc_auc_score(y, x)
n1 = np.sum(y == 1)   # positives
n0 = np.sum(y == 0)   # negatives

# Hanley–McNeil (1982) variance
Q1 = auc / (2 - auc)
Q2 = 2 * auc**2 / (1 + auc)
var = (auc*(1 - auc) + (n1 - 1)*(Q1 - auc**2) + (n0 - 1)*(Q2 - auc**2)) / (n0 * n1)
se = np.sqrt(var)

z = 1.96  # for ~95% CI
ci_lower = max(0.0, auc - z*se)
ci_upper = min(1.0, auc + z*se)

print(f"AUC = {auc:.3f} (95% CI {ci_lower:.3f}–{ci_upper:.3f})")

AUC = 0.923 (95% CI 0.851–0.995)


In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_curve, auc

#X = df['PCL_total']
X = df['PCL_reduced_total']
y = df['caps_ptsd1']

thresholds = range(1, 18) # range(19,38) # for full 20 item set
results = []

for thr in thresholds:
    y_pred = (X >= thr).astype(int)

    # confusion matrix: tn, fp, fn, tp
    tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else float('nan')  # recall
    specificity = tn / (tn + fp) if (tn + fp) > 0 else float('nan')
    ppv = tp / (tp + fp) if (tp + fp) > 0 else float('nan')
    npv = tn / (tn + fn) if (tn + fn) > 0 else float('nan')
    j_stat = sensitivity + specificity - 1

    fpr, tpr, _ = roc_curve(y, y_pred)
    roc_auc_thr = auc(fpr, tpr)

    results.append((thr, sensitivity, specificity, ppv, npv, j_stat, roc_auc_thr))

results_df = pd.DataFrame(results, columns=[
    'Threshold', 'Sensitivity', 'Specificity',
    'PPV', 'NPV', 'J_statistic', 'AUC'
])

print(results_df)
